<a href="https://colab.research.google.com/github/Midhilesh4890/RecSys/blob/main/RecommenderSystems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ratings = pd.read_csv("/content/drive/MyDrive/movie-recommendation-prodlem-dataset/ratings.csv")
movies = pd.read_csv("/content/drive/MyDrive/movie-recommendation-prodlem-dataset/movies_metadata.csv.zip")
credits = pd.read_csv('/content/drive/MyDrive/movie-recommendation-prodlem-dataset/credits.csv.zip')

<ipython-input-3-baf3cb885775>:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv("/content/drive/MyDrive/movie-recommendation-prodlem-dataset/movies_metadata.csv.zip")


In [ ]:
ratings.shape

(25000095, 4)

In [ ]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [ ]:
movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [ ]:
##Some ids have date like ids
movies.drop(movies.index[19730],inplace=True)
movies.drop(movies.index[29502],inplace=True)
movies.drop(movies.index[35585],inplace=True)

In [ ]:
ratings.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

In [ ]:
movies.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [ ]:
ratings.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

In [ ]:
movies.dtypes

adult                     object
belongs_to_collection     object
budget                    object
genres                    object
homepage                  object
id                        object
imdb_id                   object
original_language         object
original_title            object
overview                  object
popularity                object
poster_path               object
production_companies      object
production_countries      object
release_date              object
revenue                  float64
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
video                     object
vote_average             float64
vote_count               float64
dtype: object

In [ ]:
movies.id = movies.id.astype('int')

In [ ]:
type(movies.id[0])

numpy.int64

In [ ]:
ratings.movieId.isin(movies.id).sum()

10433371

In [ ]:
ratings = pd.merge(ratings,movies[['title','id']],left_on='movieId',right_on='id')
ratings.head()

,userId,movieId,rating,timestamp,title,id
0,1,296,5.0,1147880044,Terminator 3: Rise of the Machines,296
1,3,296,5.0,1439474476,Terminator 3: Rise of the Machines,296
2,4,296,4.0,1573938898,Terminator 3: Rise of the Machines,296
3,5,296,4.0,830786155,Terminator 3: Rise of the Machines,296
4,7,296,4.0,835444730,Terminator 3: Rise of the Machines,296


In [ ]:
ratings.drop(['timestamp','id'],axis=1,inplace=True)

In [ ]:
ratings.shape

(10434471, 4)

In [ ]:
ratings.sample(5)

,userId,movieId,rating,title
3600230,36323,122924,3.0,One in the Chamber
7661024,26340,339,3.0,Night on Earth
1956371,107474,5816,4.0,Waiter
5610465,150876,158,3.5,Knockin' on Heaven's Door
2749745,98449,3527,4.0,Don't Come Knocking


In [ ]:
ratings.isnull().sum()

userId     0
movieId    0
rating     0
title      0
dtype: int64

In [ ]:
ratings_count = ratings.groupby(by="title")['rating'].count().reset_index().rename(columns={'rating':'totalRatings'})[['title','totalRatings']]

In [ ]:
ratings_count

,title,totalRatings
0,!Women Art Revolution,907
1,"$1,000 on the Black",1
2,$5 a Day,139
3,'Gator Bait,199
4,'R Xmas,84
...,...,...
7807,Ödipussi,274
7808,Şaban Oğlu Şaban,129
7809,Škola princů,9
7810,Šíleně smutná princezna,294


In [ ]:
len(ratings['title'].unique())

7812

In [ ]:
ratings_count.sample(5)

,title,totalRatings
1048,Bunny Lake Is Missing,876
5737,The Clone Returns Home,40
6694,The Return of the King,1627
7293,True Love and Chaos,91
1112,Captain America: The First Avenger,349


In [ ]:
ratings_count.head()

,title,totalRatings
0,!Women Art Revolution,907
1,"$1,000 on the Black",1
2,$5 a Day,139
3,'Gator Bait,199
4,'R Xmas,84


In [ ]:
ratings_total = pd.merge(ratings,ratings_count,on='title',how='left')

In [ ]:
ratings_total.shape

(10434471, 5)

In [ ]:
ratings_total.head()

,userId,movieId,rating,title,totalRatings
0,1,296,5.0,Terminator 3: Rise of the Machines,79672
1,3,296,5.0,Terminator 3: Rise of the Machines,79672
2,4,296,4.0,Terminator 3: Rise of the Machines,79672
3,5,296,4.0,Terminator 3: Rise of the Machines,79672
4,7,296,4.0,Terminator 3: Rise of the Machines,79672


In [ ]:
ratings_count['totalRatings'].describe()

count     7812.000000
mean      1335.697773
std       4781.327645
min          1.000000
25%          6.000000
50%         38.000000
75%        394.250000
max      81482.000000
Name: totalRatings, dtype: float64

In [ ]:
ratings_count['totalRatings'].quantile(np.arange(.6,1,0.01))

0.60       85.60
0.61       94.00
0.62      103.00
0.63      112.00
0.64      124.00
0.65      137.00
0.66      150.26
0.67      167.00
0.68      185.00
0.69      200.00
0.70      219.00
0.71      244.00
0.72      281.92
0.73      312.00
0.74      349.00
0.75      394.25
0.76      440.00
0.77      503.47
0.78      565.58
0.79      636.69
0.80      720.80
0.81      834.91
0.82      958.00
0.83     1057.00
0.84     1202.00
0.85     1356.35
0.86     1550.76
0.87     1774.56
0.88     2130.68
0.89     2497.37
0.90     2899.80
0.91     3458.01
0.92     4099.24
0.93     4756.69
0.94     5788.02
0.95     7339.35
0.96     9174.56
0.97    11581.11
0.98    15457.24
0.99    23746.72
Name: totalRatings, dtype: float64

In [ ]:
votes_count_threshold = 20

In [ ]:
ratings_top = ratings_total.query('totalRatings > @votes_count_threshold')

In [ ]:
ratings_top.shape

(10414278, 5)

In [ ]:
ratings_top.head()

,userId,movieId,rating,title,totalRatings
0,1,296,5.0,Terminator 3: Rise of the Machines,79672
1,3,296,5.0,Terminator 3: Rise of the Machines,79672
2,4,296,4.0,Terminator 3: Rise of the Machines,79672
3,5,296,4.0,Terminator 3: Rise of the Machines,79672
4,7,296,4.0,Terminator 3: Rise of the Machines,79672


In [ ]:
if not ratings_top[ratings_top.duplicated(['userId','title'])].empty:
    ratings_top = ratings_top.drop_duplicates(['userId','title'])

In [ ]:
ratings_top.shape

(10381702, 5)

Reshape the data using pivot function

In [ ]:
df_for_knn = ratings_top.pivot(index='title',columns='userId',values='rating').fillna(0) 

In [ ]:
df_for_knn.head()

In [ ]:
df_for_knn.shape

Let's use sparse matrix representation of this matrix

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
df_for_knn_sparse = csr_matrix(df_for_knn.values) 

## Recommendations using KNearestNeighbors method

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
model_knn = NearestNeighbors(metric='cosine',algorithm='brute')

In [ ]:
model_knn.fit(df_for_knn_sparse)

In [ ]:
query_index = np.random.choice(df_for_knn.shape[0])

In [ ]:
distances, indices = model_knn.kneighbors(df_for_knn.loc['Batman Returns'].values.reshape(1,-1),n_neighbors=6)

In [ ]:
distances, indices = model_knn.kneighbors(df_for_knn.iloc[query_index,:].values.reshape(1,-1),n_neighbors=6)

In [ ]:
for i in range(0,len(distances.flatten())):
    if i==0:
        print("Recommendations for movie: {0}\n".format(df_for_knn.index[query_index]))
    else:
        print("{0}: {1}, with distance of {2}".format(i,df_for_knn.index[indices.flatten()[i]],distances.flatten()[i]))